# Create stock

We created a database called Supermarket.db in the previous notebook, this data base contains 2 tables: Recipe and Ingredients.In this notebook we will analyse the tables and then create a table with the stocks of a supermarket, containing the id of the product, the quantity and their expiration date.

## Analyse the __Ingredients__ table
The table is quiet small so I will import it with sql than analyse it in pandas.

### import the table

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('../Data/Supermarket.db')

query = "SELECT * FROM Ingredients"

df = pd.read_sql_query(query, conn)

conn.commit()
conn.close()

### Analyse the table

In [3]:
df['aisle'].nunique()

94

There is a total of 94 differents aisle. Ideally we would analyse them one by one to be able to have a coherent expiration date. It would also give us information about the ecological impact. Indeed, throwing 1 kg of meat as not the same impact than 1 salad. However this would take an incredible amount of time and the goal here is not necesseraly to have an app working in real life, I would need a real database to do it.

In [4]:
df[df['name'].duplicated()] #all names are unique too

,id,aisle,name


In [5]:
#will be used later
ingr_id = df['id'].unique()

## Analyse the __Recipes__ table

### import the table

In [6]:
conn = sqlite3.connect('../Data/Supermarket.db')

query = "SELECT * FROM Recipes"

df = pd.read_sql_query(query, conn)

conn.commit()
conn.close()

### Analyse it

We would like to have informations on what units of measures are used.

In [7]:
import ast

def get_unit(ingredients: str) -> list:
    """ 
    ingredients is a list of dictionnaries but is has the type of a string.
    for each dictionnaries contained in ingredients, keep only the keys: unit
    return the cleaned list of dictionnaries.
    """
    ingredients = ast.literal_eval(ingredients) #the column contains string, we want it to be a list
    cleaned_list = []

    for ingredient in ingredients:
        cleaned_dict = {key: value for key, value in ingredient.items() if key in ['unit']}
        cleaned_list.append(cleaned_dict)

    return cleaned_list

In [8]:
all_units_duplicate = df['ingredients'].apply(get_unit)


list_unit = []

for recipe in all_units_duplicate:
    for unit in recipe:
        if unit['unit'] in list_unit:
            continue
        else:
            list_unit.append(unit['unit'])

In [9]:
len(list_unit)

161

There are a total of 162 differents unit. Some are the same (tablespoon = tablespoon), some are unknown to me, some are empty,... Similarly to above, cleaning it to be able to have a coherent result would take a lot of time. I have so decided that the default value of quantity is 1 and that there is no units.

It means that every time a recipe use an ingredient it will use quantity = 1 of this ingedient.

## Create the table Stocks
We will now buid a table representing the stocks of a supermarket. 'Expiration date' will be replace by 'urgent', whic is an int between 1 and 5 (inluded), with 5 representing a product that is really close to its expiration date, 1 very far.

__TABLE Stocks__:

| ingredient_id| urgent          | quantity|
|--------------|-----------------|---------|
| PRIMARY KEY  | PRIMARY KEY     |         |

We have 1235 differents ingredients, the stocks will be created the following way:

* 300 lines of urgent = 5, quantity random(1,2), ingredient_id = random
* 500 lines of urgent = 4, quantity random(1,3), ingredient_id = random
* 800 lines of urgent = 3, quantity random(1,4), ingredient_id = random
* 800 lines of urgent = 2, quantity random(1,4), ingredient_id = random
* 600 lines of urgent = 1, quantity random(1,3), ingredient_id = random

In [10]:
import pandas as pd
import numpy as np

data = []

data.extend([(id, 5, np.random.choice([1, 2])) for id in np.random.choice(ingr_id, size=300, replace=True)])
data.extend([(id, 4, np.random.randint(1, 4)) for id in np.random.choice(ingr_id, size=500, replace=True)])
data.extend([(id, 3, np.random.randint(1, 5)) for id in np.random.choice(ingr_id, size=800, replace=True)])
data.extend([(id, 2, np.random.randint(1, 5)) for id in np.random.choice(ingr_id, size=800, replace=True)])
data.extend([(id, 1, np.random.randint(1, 4)) for id in np.random.choice(ingr_id, size=600, replace=True)])

df_stocks = pd.DataFrame(data, columns=['ingredient_id', 'urgent', 'quantity'])



In [11]:
conn = sqlite3.connect('../Data/Supermarket.db')

df_stocks.to_sql('Stocks', conn, if_exists='replace', index=False)

conn.commit()
conn.close()